In [2]:
df = spark.sql("SELECT * FROM bing_news_lakehouse.tbl_latestnews")
display(df)

StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 309ad03c-3348-4963-8379-f4e63a4db4c3)

In [3]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 4, Finished, Available)

In [4]:
#Building the model, by setting column to perform sentiment analysis , kind of analysis, 
# Response column for output of analysis , Error column to handle errors
model =(AnalyzeText().setTextCol("description").setKind("SentimentAnalysis")
                     .setOutputCol("response").setErrorCol("error"))

StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 5, Finished, Available)

In [5]:
#applying the model to our dataframe and assigning the model to response df
response=model.transform(df)

StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 6, Finished, Available)

In [6]:
display(response)

StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 007603aa-2917-4a6f-8ac9-70f07165a244)

creating sentiment column to extract sentiment from the response column 

In [7]:
from pyspark.sql.functions import col
senti=response.withColumn('sentiment',col("response.documents.sentiment"))

StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 8, Finished, Available)

In [8]:
display(senti)

StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 54582d60-b8c2-4ef7-b7da-da3e85e23ace)

Dropping unncecessary columns like error and response

In [9]:
sentiment= senti.drop("response","error")

StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 10, Finished, Available)

### Finally updating the data into new table and setting INCREMENTAL LOAD

In [11]:
from pyspark.sql.utils import AnalysisException

try:
    name="bing_news_lakehouse.tbl_sentimentnews"
    sentiment.write.format("delta").saveAsTable(name)
except AnalysisException:
    print('Table Already Exists')
    sentiment.createOrReplaceTempView("viewOfsentimentdf")

    spark.sql(f""" MERGE INTO {name} target_table
                USING viewOfsentimentdf source_table

                ON target_table.articleurl=source_table.articleurl

                WHEN MATCHED AND 

                source_table.title <> target_table.title OR
                source_table.category <> target_table.category OR
                source_table.description <> target_table.description OR
                source_table.provider <> target_table.provider OR
                source_table.imageurl <> target_table.imageurl OR
                source_table.dateposted <> target_table.dateposted

                THEN UPDATE SET * 

                WHEN NOT MATCHED THEN INSERT * 
             """)



StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 12, Finished, Available)

Table Already Exists


In [12]:
display(sentiment)

StatementMeta(, 04413299-d1c9-43fd-9705-e45f13631c2b, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 571db8e3-c66a-4023-be6b-3088fcd135c8)